# 🧹 Làm Sạch Dữ Liệu - Movie Ticket Company Analysis

## 📖 Tổng quan dự án

Dự án phân tích dữ liệu của **công ty bán vé xem phim trực tuyến** với mục tiêu:
- Hiểu hành vi khách hàng trong quá trình đặt vé
- Đánh giá hiệu quả các chiến dịch marketing  
- Xây dựng dashboard trực quan cho bộ phận marketing và vận hành

## 🗂️ Cấu trúc dữ liệu (Star Schema)

**Fact Table:**
- `ticket_history.csv` - Lịch sử đặt vé và giao dịch

**Dimension Tables:**
- `customer.csv` - Thông tin khách hàng (ID, giới tính, ngày sinh)
- `campaign.csv` - Chiến dịch marketing (ID, loại chiến dịch)
- `device_detail.csv` - Thiết bị truy cập
- `status_detail.csv` - Trạng thái ticket

---


## 🎯 Mục tiêu làm sạch dữ liệu

### 1. **Data Quality Checks**
- ✅ Kiểm tra missing values
- ✅ Kiểm tra duplicates  
- ✅ Kiểm tra data types
- ✅ Kiểm tra outliers

### 2. **Data Consistency**
- 🔄 Chuẩn hóa định dạng ngày tháng
- 🔄 Kiểm tra foreign key relationships
- 🔄 Đảm bảo tính nhất quán của categorical data

### 3. **Business Logic Validation**
- 🔄 Kiểm tra giá vé hợp lý (final_price = original_price - discount_value)
- 🔄 Kiểm tra thời gian hợp lệ
- 🔄 Kiểm tra trạng thái ticket logic

---


# 📊 PHASE 1: Data Discovery & Assessment

## Mục tiêu Phase 1
- Hiểu cấu trúc và chất lượng dữ liệu hiện tại
- Xác định các vấn đề cần xử lý
- Lập kế hoạch chi tiết cho các phase tiếp theo

---


In [14]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Đã import thành công các thư viện cần thiết")
print(f"📅 Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)


✅ Đã import thành công các thư viện cần thiết
📅 Thời gian bắt đầu: 2025-10-04 17:56:11


In [15]:
# Load dữ liệu từ các file CSV
print("📂 Đang load dữ liệu từ các file CSV...")

try:
    # Load các bảng dimension
    df_customer = pd.read_csv('../data/customer.csv')
    df_campaign = pd.read_csv('../data/campaign.csv') 
    df_device = pd.read_csv('../data/device_detail.csv')
    df_status = pd.read_csv('../data/status_detail.csv')
    
    # Load bảng fact chính
    df_ticket = pd.read_csv('../data/ticket_history.csv')
    
    print("✅ Load dữ liệu thành công!")
    
except Exception as e:
    print(f"❌ Lỗi khi load dữ liệu: {e}")
    raise


📂 Đang load dữ liệu từ các file CSV...
✅ Load dữ liệu thành công!


## 1️⃣ Thông tin cơ bản về dữ liệu

Kiểm tra kích thước, cấu trúc và cột của từng bảng dữ liệu


In [16]:
# Thông tin tổng quan về dữ liệu
datasets = {
    '📋 Customer (Dimension)': df_customer,
    '📢 Campaign (Dimension)': df_campaign,
    '📱 Device (Dimension)': df_device, 
    '🎫 Status (Dimension)': df_status,
    '🎬 Ticket History (Fact)': df_ticket
}

print("📊 THÔNG TIN TỔNG QUAN VỀ DỮ LIỆU")
print("="*60)

for name, df in datasets.items():
    print(f"\n{name}")
    print(f"   📏 Kích thước: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"   📝 Các cột: {list(df.columns)}")
    
print(f"\n🔢 TỔNG CỘNG: {sum(df.shape[0] for df in datasets.values()):,} records")


📊 THÔNG TIN TỔNG QUAN VỀ DỮ LIỆU

📋 Customer (Dimension)
   📏 Kích thước: 131,400 rows × 3 columns
   📝 Các cột: ['customer_id', 'usergender', 'dob']

📢 Campaign (Dimension)
   📏 Kích thước: 216 rows × 2 columns
   📝 Các cột: ['campaign_id', 'campaign_type']

📱 Device (Dimension)
   📏 Kích thước: 139,902 rows × 3 columns
   📝 Các cột: ['device_number', 'model', 'platform']

🎫 Status (Dimension)
   📏 Kích thước: 8 rows × 3 columns
   📝 Các cột: ['status_id', 'description', 'error_group']

🎬 Ticket History (Fact)
   📏 Kích thước: 154,827 rows × 12 columns
   📝 Các cột: ['ticket_id', 'customer_id', 'paying_method', 'theater_name', 'device_number', 'original_price', 'discount_value', 'final_price', 'time', 'status_id', 'campaign_id', 'movie_name']

🔢 TỔNG CỘNG: 426,353 records


## 2️⃣ Kiểm tra Missing Values

Phân tích tỷ lệ giá trị thiếu trong từng cột của mỗi bảng


In [17]:
# Phân tích Missing Values
print("🔍 PHÂN TÍCH MISSING VALUES")
print("="*60)

total_missing_issues = 0

for name, df in datasets.items():
    print(f"\n{name}")
    print("-" * 40)
    
    # Tính missing values
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    
    # Tạo bảng tổng hợp
    missing_df = pd.DataFrame({
        'Cột': missing_data.index,
        'Số lượng thiếu': missing_data.values,
        'Tỷ lệ (%)': missing_percent.values
    })
    
    # Chỉ hiển thị các cột có missing values
    missing_df = missing_df[missing_df['Số lượng thiếu'] > 0].sort_values('Số lượng thiếu', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df.to_string(index=False))
        total_missing_issues += len(missing_df)
    else:
        print("✅ Không có missing values")

print(f"\n📊 TỔNG KẾT: {total_missing_issues} cột có missing values")


🔍 PHÂN TÍCH MISSING VALUES

📋 Customer (Dimension)
----------------------------------------
✅ Không có missing values

📢 Campaign (Dimension)
----------------------------------------
✅ Không có missing values

📱 Device (Dimension)
----------------------------------------
          Cột  Số lượng thiếu  Tỷ lệ (%)
        model            7139   5.102858
device_number               1   0.000715

🎫 Status (Dimension)
----------------------------------------
        Cột  Số lượng thiếu  Tỷ lệ (%)
error_group               1       12.5

🎬 Ticket History (Fact)
----------------------------------------
✅ Không có missing values

📊 TỔNG KẾT: 3 cột có missing values


## 3️⃣ Kiểm tra Duplicates

Tìm và đếm các bản ghi trùng lặp trong từng bảng


In [18]:
# Kiểm tra Duplicates
print("🔍 PHÂN TÍCH DUPLICATES")
print("="*60)

total_duplicate_issues = 0

for name, df in datasets.items():
    print(f"\n{name}")
    print("-" * 40)
    
    # Kiểm tra duplicate rows
    duplicate_rows = df.duplicated().sum()
    duplicate_percent = (duplicate_rows / len(df)) * 100
    
    print(f"📊 Duplicate rows: {duplicate_rows:,} ({duplicate_percent:.2f}%)")
    
    # Kiểm tra duplicate trong primary key (nếu có)
    if name == '📋 Customer (Dimension)':
        customer_duplicates = df['customer_id'].duplicated().sum()
        print(f"🔑 Duplicate customer_id: {customer_duplicates}")
        if customer_duplicates > 0:
            total_duplicate_issues += customer_duplicates
            
    elif name == '📢 Campaign (Dimension)':
        campaign_duplicates = df['campaign_id'].duplicated().sum()
        print(f"🔑 Duplicate campaign_id: {campaign_duplicates}")
        if campaign_duplicates > 0:
            total_duplicate_issues += campaign_duplicates
            
    elif name == '🎫 Status (Dimension)':
        status_duplicates = df['status_id'].duplicated().sum()
        print(f"🔑 Duplicate status_id: {status_duplicates}")
        if status_duplicates > 0:
            total_duplicate_issues += status_duplicates
            
    elif name == '🎬 Ticket History (Fact)':
        ticket_duplicates = df['ticket_id'].duplicated().sum()
        print(f"🔑 Duplicate ticket_id: {ticket_duplicates}")
        if ticket_duplicates > 0:
            total_duplicate_issues += ticket_duplicates
    
    if duplicate_rows == 0 and 'duplicate' not in locals():
        print("✅ Không có duplicates")

print(f"\n📊 TỔNG KẾT: {total_duplicate_issues} duplicate primary keys")


🔍 PHÂN TÍCH DUPLICATES

📋 Customer (Dimension)
----------------------------------------
📊 Duplicate rows: 0 (0.00%)
🔑 Duplicate customer_id: 0
✅ Không có duplicates

📢 Campaign (Dimension)
----------------------------------------
📊 Duplicate rows: 0 (0.00%)
🔑 Duplicate campaign_id: 0
✅ Không có duplicates

📱 Device (Dimension)
----------------------------------------
📊 Duplicate rows: 0 (0.00%)
✅ Không có duplicates

🎫 Status (Dimension)
----------------------------------------
📊 Duplicate rows: 0 (0.00%)
🔑 Duplicate status_id: 0
✅ Không có duplicates

🎬 Ticket History (Fact)
----------------------------------------
📊 Duplicate rows: 102 (0.07%)
🔑 Duplicate ticket_id: 102

📊 TỔNG KẾT: 102 duplicate primary keys


## 4️⃣ Kiểm tra Data Types

Phân tích kiểu dữ liệu hiện tại và xác định cần chuyển đổi


In [19]:
# Kiểm tra Data Types
print("🔍 PHÂN TÍCH DATA TYPES")
print("="*60)

for name, df in datasets.items():
    print(f"\n{name}")
    print("-" * 40)
    
    # Hiển thị data types
    dtype_df = pd.DataFrame({
        'Cột': df.columns,
        'Data Type': df.dtypes.values,
        'Non-null Count': df.count().values,
        'Memory Usage': [df[col].memory_usage(deep=True) for col in df.columns]
    })
    
    print(dtype_df.to_string(index=False))
    
    # Kiểm tra các cột có thể cần chuyển đổi
    date_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['date', 'time', 'dob'])]
    numeric_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['price', 'value', 'id'])]
    
    if date_columns:
        print(f"\n📅 Cột có thể là date: {date_columns}")
    if numeric_columns:
        print(f"🔢 Cột có thể là numeric: {numeric_columns}")

print(f"\n✅ Hoàn thành phân tích data types")


🔍 PHÂN TÍCH DATA TYPES

📋 Customer (Dimension)
----------------------------------------
        Cột Data Type  Non-null Count  Memory Usage
customer_id     int64          131400       1051332
 usergender    object          131400       7172826
        dob    object          131400       7602907

📅 Cột có thể là date: ['dob']
🔢 Cột có thể là numeric: ['customer_id']

📢 Campaign (Dimension)
----------------------------------------
          Cột Data Type  Non-null Count  Memory Usage
  campaign_id     int64             216          1860
campaign_type    object             216         12814
🔢 Cột có thể là numeric: ['campaign_id']

📱 Device (Dimension)
----------------------------------------
          Cột Data Type  Non-null Count  Memory Usage
device_number    object          139901      11332145
        model    object          132763       8158048
     platform    object          139902       7711556

🎫 Status (Dimension)
----------------------------------------
        Cột Data Type 

# 📊 PHASE 2: Data Type Conversion & Business Logic Validation

## Mục tiêu Phase 2
- Chuyển đổi data types phù hợp
- Kiểm tra business logic (giá vé, thời gian)
- Xử lý missing values đã phát hiện

## 🎯 Chiến lược xử lý Missing Values:
1. **Device.model (7,139 missing - 5.1%)**: Có thể tạo category "Unknown" hoặc drop records
2. **Device.device_number (1 missing)**: Drop record này vì là primary key
3. **Status.error_group (1 missing - 12.5%)**: Có thể fill bằng "no_error" cho status thành công

---


## 1️⃣ Chuyển đổi Data Types

Chuyển đổi các cột date và numeric về đúng định dạng


In [20]:
# Chuyển đổi Data Types
print("🔄 CHUYỂN ĐỔI DATA TYPES")
print("="*60)

# Lưu trữ thông tin trước khi chuyển đổi
print("📊 THÔNG TIN TRƯỚC KHI CHUYỂN ĐỔI:")
print(f"Customer dob type: {df_customer['dob'].dtype}")
print(f"Ticket time type: {df_ticket['time'].dtype}")
print(f"Ticket original_price type: {df_ticket['original_price'].dtype}")

try:
    # 1. Chuyển đổi ngày sinh (dob) - Customer table
    print("\n🔄 Đang chuyển đổi dob...")
    df_customer['dob'] = pd.to_datetime(df_customer['dob'], format='%m/%d/%Y', errors='coerce')
    
    # 2. Chuyển đổi thời gian đặt vé (time) - Ticket table  
    print("🔄 Đang chuyển đổi time...")
    df_ticket['time'] = pd.to_datetime(df_ticket['time'], errors='coerce')
    
    # 3. Chuyển đổi các cột giá - Ticket table
    print("🔄 Đang chuyển đổi price columns...")
    price_columns = ['original_price', 'discount_value', 'final_price']
    for col in price_columns:
        df_ticket[col] = pd.to_numeric(df_ticket[col], errors='coerce')
    
    print("\n✅ CHUYỂN ĐỔI THÀNH CÔNG!")
    
except Exception as e:
    print(f"❌ Lỗi khi chuyển đổi: {e}")
    raise

print("\n📊 THÔNG TIN SAU KHI CHUYỂN ĐỔI:")
print(f"Customer dob type: {df_customer['dob'].dtype}")
print(f"Ticket time type: {df_ticket['time'].dtype}")
print(f"Ticket original_price type: {df_ticket['original_price'].dtype}")


🔄 CHUYỂN ĐỔI DATA TYPES
📊 THÔNG TIN TRƯỚC KHI CHUYỂN ĐỔI:
Customer dob type: object
Ticket time type: object
Ticket original_price type: float64

🔄 Đang chuyển đổi dob...
🔄 Đang chuyển đổi time...
🔄 Đang chuyển đổi price columns...

✅ CHUYỂN ĐỔI THÀNH CÔNG!

📊 THÔNG TIN SAU KHI CHUYỂN ĐỔI:
Customer dob type: datetime64[ns]
Ticket time type: datetime64[ns]
Ticket original_price type: float64


## 2️⃣ Kiểm tra Business Logic - Giá vé

Kiểm tra công thức: final_price = original_price - discount_value


In [21]:
# Kiểm tra Business Logic - Giá vé
print("💰 KIỂM TRA BUSINESS LOGIC - GIÁ VÉ")
print("="*60)

# Tính toán giá vé theo công thức
df_ticket['calculated_final_price'] = df_ticket['original_price'] - df_ticket['discount_value']

# Kiểm tra sự khác biệt (cho phép sai số nhỏ do floating point)
tolerance = 0.01  # 1 cent
df_ticket['price_diff'] = abs(df_ticket['final_price'] - df_ticket['calculated_final_price'])
df_ticket['price_match'] = df_ticket['price_diff'] <= tolerance

# Thống kê
total_tickets = len(df_ticket)
matching_prices = df_ticket['price_match'].sum()
non_matching_prices = total_tickets - matching_prices

print(f"📊 TỔNG SỐ TICKETS: {total_tickets:,}")
print(f"✅ Giá vé đúng logic: {matching_prices:,} ({matching_prices/total_tickets*100:.2f}%)")
print(f"❌ Giá vé không đúng logic: {non_matching_prices:,} ({non_matching_prices/total_tickets*100:.2f}%)")

# Hiển thị một vài ví dụ không đúng logic
if non_matching_prices > 0:
    print(f"\n🔍 VÍ DỤ CÁC TICKET KHÔNG ĐÚNG LOGIC:")
    wrong_prices = df_ticket[~df_ticket['price_match']].head(5)
    print(wrong_prices[['ticket_id', 'original_price', 'discount_value', 'final_price', 'calculated_final_price', 'price_diff']].to_string(index=False))

# Kiểm tra các giá trị âm hoặc không hợp lý
print(f"\n🔍 KIỂM TRA GIÁ TRỊ KHÔNG HỢP LÝ:")
print(f"Original price âm: {(df_ticket['original_price'] < 0).sum()}")
print(f"Discount value âm: {(df_ticket['discount_value'] < 0).sum()}")
print(f"Final price âm: {(df_ticket['final_price'] < 0).sum()}")
print(f"Discount > Original price: {(df_ticket['discount_value'] > df_ticket['original_price']).sum()}")


💰 KIỂM TRA BUSINESS LOGIC - GIÁ VÉ
📊 TỔNG SỐ TICKETS: 154,827
✅ Giá vé đúng logic: 154,827 (100.00%)
❌ Giá vé không đúng logic: 0 (0.00%)

🔍 KIỂM TRA GIÁ TRỊ KHÔNG HỢP LÝ:
Original price âm: 0
Discount value âm: 0
Final price âm: 0
Discount > Original price: 0


## 3️⃣ Kiểm tra Foreign Key Relationships

Kiểm tra tính nhất quán giữa các bảng dimension và fact


In [22]:
# Kiểm tra Foreign Key Relationships
print("🔗 KIỂM TRA FOREIGN KEY RELATIONSHIPS")
print("="*60)

# 1. Kiểm tra customer_id
print("👥 KIỂM TRA CUSTOMER_ID:")
customer_in_ticket = df_ticket['customer_id'].nunique()
customer_in_dim = df_customer['customer_id'].nunique()
customer_orphan = df_ticket[~df_ticket['customer_id'].isin(df_customer['customer_id'])]['customer_id'].nunique()

print(f"   Unique customer_id trong ticket: {customer_in_ticket:,}")
print(f"   Unique customer_id trong customer dim: {customer_in_dim:,}")
print(f"   Orphan customer_id: {customer_orphan:,}")

# 2. Kiểm tra campaign_id (trừ campaign_id = 0)
print(f"\n📢 KIỂM TRA CAMPAIGN_ID:")
campaign_in_ticket = df_ticket[df_ticket['campaign_id'] != 0]['campaign_id'].nunique()
campaign_in_dim = df_campaign['campaign_id'].nunique()
campaign_orphan = df_ticket[(df_ticket['campaign_id'] != 0) & (~df_ticket['campaign_id'].isin(df_campaign['campaign_id']))]['campaign_id'].nunique()
campaign_zero_count = (df_ticket['campaign_id'] == 0).sum()

print(f"   Unique campaign_id trong ticket (khác 0): {campaign_in_ticket:,}")
print(f"   Unique campaign_id trong campaign dim: {campaign_in_dim:,}")
print(f"   Orphan campaign_id: {campaign_orphan:,}")
print(f"   Campaign_id = 0: {campaign_zero_count:,}")

# 3. Kiểm tra device_number
print(f"\n📱 KIỂM TRA DEVICE_NUMBER:")
device_in_ticket = df_ticket['device_number'].nunique()
device_in_dim = df_device['device_number'].nunique()
device_orphan = df_ticket[~df_ticket['device_number'].isin(df_device['device_number'])]['device_number'].nunique()

print(f"   Unique device_number trong ticket: {device_in_ticket:,}")
print(f"   Unique device_number trong device dim: {device_in_dim:,}")
print(f"   Orphan device_number: {device_orphan:,}")

# 4. Kiểm tra status_id
print(f"\n🎫 KIỂM TRA STATUS_ID:")
status_in_ticket = df_ticket['status_id'].nunique()
status_in_dim = df_status['status_id'].nunique()
status_orphan = df_ticket[~df_ticket['status_id'].isin(df_status['status_id'])]['status_id'].nunique()

print(f"   Unique status_id trong ticket: {status_in_ticket:,}")
print(f"   Unique status_id trong status dim: {status_in_dim:,}")
print(f"   Orphan status_id: {status_orphan:,}")

# Hiển thị các orphan values nếu có
if status_orphan > 0:
    print(f"\n🔍 Orphan status_id values:")
    orphan_status = df_ticket[~df_ticket['status_id'].isin(df_status['status_id'])]['status_id'].unique()
    print(f"   {orphan_status}")


🔗 KIỂM TRA FOREIGN KEY RELATIONSHIPS
👥 KIỂM TRA CUSTOMER_ID:
   Unique customer_id trong ticket: 119,477
   Unique customer_id trong customer dim: 131,400
   Orphan customer_id: 0

📢 KIỂM TRA CAMPAIGN_ID:
   Unique campaign_id trong ticket (khác 0): 210
   Unique campaign_id trong campaign dim: 216
   Orphan campaign_id: 0
   Campaign_id = 0: 63,129

📱 KIỂM TRA DEVICE_NUMBER:
   Unique device_number trong ticket: 126,459
   Unique device_number trong device dim: 139,901
   Orphan device_number: 1

🎫 KIỂM TRA STATUS_ID:
   Unique status_id trong ticket: 8
   Unique status_id trong status dim: 8
   Orphan status_id: 0


# 🧹 PHASE 3: Data Cleaning & Final Preparation

## Mục tiêu Phase 3
- Xử lý tất cả vấn đề đã phát hiện
- Chuẩn hóa dữ liệu cho phân tích
- Export clean datasets

## 🎯 Chiến lược xử lý:
1. **Duplicates**: Drop 102 duplicate tickets (giữ record đầu tiên)
2. **Missing Values**: 
   - Device.model → "Unknown" 
   - Device.device_number → Drop 1 record
   - Status.error_group → "no_error"
3. **Campaign Enhancement**: Tạo "No Campaign" record cho campaign_id = 0
4. **Orphan Records**: Drop 1 orphan device ticket

---


## 1️⃣ Xử lý Duplicate Tickets

Loại bỏ 102 duplicate ticket_id, giữ lại record đầu tiên


In [23]:
# Xử lý Duplicate Tickets
print("🔄 XỬ LÝ DUPLICATE TICKETS")
print("="*60)

# Lưu số lượng trước khi xử lý
before_count = len(df_ticket)
duplicate_count = df_ticket['ticket_id'].duplicated().sum()

print(f"📊 Trước khi xử lý:")
print(f"   Tổng tickets: {before_count:,}")
print(f"   Duplicate tickets: {duplicate_count:,}")

# Xóa duplicates, giữ lại record đầu tiên
df_ticket = df_ticket.drop_duplicates(subset=['ticket_id'], keep='first')

after_count = len(df_ticket)
removed_count = before_count - after_count

print(f"\n📊 Sau khi xử lý:")
print(f"   Tổng tickets: {after_count:,}")
print(f"   Đã xóa: {removed_count:,} duplicates")

# Xóa các cột tạm thời đã tạo trong Phase 2
df_ticket = df_ticket.drop(['calculated_final_price', 'price_diff', 'price_match'], axis=1, errors='ignore')

print(f"\n✅ Hoàn thành xử lý duplicates")


🔄 XỬ LÝ DUPLICATE TICKETS
📊 Trước khi xử lý:
   Tổng tickets: 154,827
   Duplicate tickets: 102

📊 Sau khi xử lý:
   Tổng tickets: 154,725
   Đã xóa: 102 duplicates

✅ Hoàn thành xử lý duplicates


## 2️⃣ Xử lý Missing Values

Xử lý các missing values đã phát hiện


In [24]:
# Xử lý Missing Values
print("🔄 XỬ LÝ MISSING VALUES")
print("="*60)

# 1. Xử lý Device.model missing values
print("📱 XỬ LÝ DEVICE.MODEL:")
model_missing_before = df_device['model'].isnull().sum()
print(f"   Missing trước: {model_missing_before:,}")

# Fill missing model với "Unknown"
df_device['model'] = df_device['model'].fillna('Unknown')

model_missing_after = df_device['model'].isnull().sum()
print(f"   Missing sau: {model_missing_after}")
print(f"   ✅ Đã fill {model_missing_before:,} missing values với 'Unknown'")

# 2. Xử lý Device.device_number missing (drop record)
print(f"\n📱 XỬ LÝ DEVICE.DEVICE_NUMBER:")
device_missing_before = df_device['device_number'].isnull().sum()
print(f"   Missing trước: {device_missing_before}")

# Drop record có device_number missing
df_device = df_device.dropna(subset=['device_number'])

device_missing_after = df_device['device_number'].isnull().sum()
print(f"   Missing sau: {device_missing_after}")
print(f"   ✅ Đã drop 1 record có device_number missing")

# 3. Xử lý Status.error_group missing
print(f"\n🎫 XỬ LÝ STATUS.ERROR_GROUP:")
error_missing_before = df_status['error_group'].isnull().sum()
print(f"   Missing trước: {error_missing_before}")

# Fill missing error_group với "no_error" (cho status thành công)
df_status['error_group'] = df_status['error_group'].fillna('no_error')

error_missing_after = df_status['error_group'].isnull().sum()
print(f"   Missing sau: {error_missing_after}")
print(f"   ✅ Đã fill 1 missing value với 'no_error'")

print(f"\n📊 TỔNG KẾT XỬ LÝ MISSING VALUES:")
print(f"   Device.model: {model_missing_before:,} → {model_missing_after}")
print(f"   Device.device_number: {device_missing_before} → {device_missing_after}")
print(f"   Status.error_group: {error_missing_before} → {error_missing_after}")


🔄 XỬ LÝ MISSING VALUES
📱 XỬ LÝ DEVICE.MODEL:
   Missing trước: 7,139
   Missing sau: 0
   ✅ Đã fill 7,139 missing values với 'Unknown'

📱 XỬ LÝ DEVICE.DEVICE_NUMBER:
   Missing trước: 1
   Missing sau: 0
   ✅ Đã drop 1 record có device_number missing

🎫 XỬ LÝ STATUS.ERROR_GROUP:
   Missing trước: 1
   Missing sau: 0
   ✅ Đã fill 1 missing value với 'no_error'

📊 TỔNG KẾT XỬ LÝ MISSING VALUES:
   Device.model: 7,139 → 0
   Device.device_number: 1 → 0
   Status.error_group: 1 → 0


## 3️⃣ Tạo "No Campaign" Record

Thêm campaign record cho campaign_id = 0


In [25]:
# Tạo "No Campaign" Record
print("📢 TẠO 'NO CAMPAIGN' RECORD")
print("="*60)

# Kiểm tra campaign_id = 0 trong ticket
campaign_zero_count = (df_ticket['campaign_id'] == 0).sum()
print(f"📊 Tickets với campaign_id = 0: {campaign_zero_count:,}")

# Kiểm tra xem đã có campaign_id = 0 chưa
has_zero_campaign = (df_campaign['campaign_id'] == 0).any()
print(f"📊 Campaign table đã có campaign_id = 0: {has_zero_campaign}")

if not has_zero_campaign:
    # Tạo record "No Campaign"
    new_campaign = pd.DataFrame({
        'campaign_id': [0],
        'campaign_type': ['no campaign']
    })
    
    # Thêm vào campaign table
    df_campaign = pd.concat([df_campaign, new_campaign], ignore_index=True)
    df_campaign = df_campaign.sort_values('campaign_id').reset_index(drop=True)
    
    print(f"✅ Đã tạo 'No Campaign' record:")
    print(f"   campaign_id: 0")
    print(f"   campaign_type: 'no campaign'")
else:
    print(f"✅ Campaign table đã có record campaign_id = 0")

print(f"\n📊 Campaign table sau khi xử lý:")
print(f"   Tổng campaigns: {len(df_campaign):,}")
print(f"   Campaign types: {df_campaign['campaign_type'].unique()}")


📢 TẠO 'NO CAMPAIGN' RECORD
📊 Tickets với campaign_id = 0: 63,098
📊 Campaign table đã có campaign_id = 0: False
✅ Đã tạo 'No Campaign' record:
   campaign_id: 0
   campaign_type: 'no campaign'

📊 Campaign table sau khi xử lý:
   Tổng campaigns: 217
   Campaign types: ['no campaign' 'voucher' 'reward point' 'direct discount']


## 4️⃣ Xử lý Orphan Device Records

Loại bỏ tickets có device_number không tồn tại trong device dimension


In [26]:
# Xử lý Orphan Device Records
print("🔗 XỬ LÝ ORPHAN DEVICE RECORDS")
print("="*60)

# Kiểm tra orphan device_number
valid_devices = set(df_device['device_number'])
ticket_devices = set(df_ticket['device_number'])

orphan_devices = ticket_devices - valid_devices
orphan_count = len(orphan_devices)

print(f"📊 Orphan device_number trong tickets: {orphan_count}")

if orphan_count > 0:
    print(f"🔍 Orphan device_number values: {list(orphan_devices)}")
    
    # Đếm tickets bị ảnh hưởng
    affected_tickets = df_ticket[df_ticket['device_number'].isin(orphan_devices)]
    affected_count = len(affected_tickets)
    
    print(f"📊 Tickets bị ảnh hưởng: {affected_count}")
    
    # Drop tickets có orphan device_number
    before_count = len(df_ticket)
    df_ticket = df_ticket[~df_ticket['device_number'].isin(orphan_devices)]
    after_count = len(df_ticket)
    removed_count = before_count - after_count
    
    print(f"✅ Đã xóa {removed_count} tickets có orphan device_number")
else:
    print(f"✅ Không có orphan device records")

print(f"\n📊 Ticket table sau khi xử lý:")
print(f"   Tổng tickets: {len(df_ticket):,}")


🔗 XỬ LÝ ORPHAN DEVICE RECORDS
📊 Orphan device_number trong tickets: 1
🔍 Orphan device_number values: ['d41d8cd98f00b204e9800998ecf8427e']
📊 Tickets bị ảnh hưởng: 78
✅ Đã xóa 78 tickets có orphan device_number

📊 Ticket table sau khi xử lý:
   Tổng tickets: 154,647


## 5️⃣ Final Quality Check

Kiểm tra chất lượng dữ liệu sau khi làm sạch


In [27]:
# Final Quality Check
print("🔍 FINAL QUALITY CHECK")
print("="*60)

# Cập nhật datasets dictionary
datasets_cleaned = {
    '📋 Customer (Cleaned)': df_customer,
    '📢 Campaign (Cleaned)': df_campaign,
    '📱 Device (Cleaned)': df_device, 
    '🎫 Status (Cleaned)': df_status,
    '🎬 Ticket History (Cleaned)': df_ticket
}

print("📊 THÔNG TIN SAU KHI LÀM SẠCH:")
print("-" * 40)

for name, df in datasets_cleaned.items():
    print(f"\n{name}")
    print(f"   📏 Kích thước: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Kiểm tra missing values
    missing_count = df.isnull().sum().sum()
    if missing_count > 0:
        print(f"   ⚠️  Missing values: {missing_count}")
    else:
        print(f"   ✅ Missing values: 0")

# Kiểm tra duplicates
print(f"\n🔍 KIỂM TRA DUPLICATES:")
for name, df in datasets_cleaned.items():
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        print(f"   ⚠️  {name}: {duplicate_count} duplicates")
    else:
        print(f"   ✅ {name}: No duplicates")

# Kiểm tra Foreign Key relationships
print(f"\n🔗 KIỂM TRA FOREIGN KEYS:")
customer_orphan = df_ticket[~df_ticket['customer_id'].isin(df_customer['customer_id'])]['customer_id'].nunique()
campaign_orphan = df_ticket[(df_ticket['campaign_id'] != 0) & (~df_ticket['campaign_id'].isin(df_campaign['campaign_id']))]['campaign_id'].nunique()
device_orphan = df_ticket[~df_ticket['device_number'].isin(df_device['device_number'])]['device_number'].nunique()
status_orphan = df_ticket[~df_ticket['status_id'].isin(df_status['status_id'])]['status_id'].nunique()

print(f"   ✅ Customer orphan: {customer_orphan}")
print(f"   ✅ Campaign orphan: {campaign_orphan}")
print(f"   ✅ Device orphan: {device_orphan}")
print(f"   ✅ Status orphan: {status_orphan}")

print(f"\n🎉 DỮ LIỆU ĐÃ SẠCH VÀ SẴN SÀNG CHO PHÂN TÍCH!")


🔍 FINAL QUALITY CHECK
📊 THÔNG TIN SAU KHI LÀM SẠCH:
----------------------------------------

📋 Customer (Cleaned)
   📏 Kích thước: 131,400 rows × 3 columns
   ✅ Missing values: 0

📢 Campaign (Cleaned)
   📏 Kích thước: 217 rows × 2 columns
   ✅ Missing values: 0

📱 Device (Cleaned)
   📏 Kích thước: 139,901 rows × 3 columns
   ✅ Missing values: 0

🎫 Status (Cleaned)
   📏 Kích thước: 8 rows × 3 columns
   ✅ Missing values: 0

🎬 Ticket History (Cleaned)
   📏 Kích thước: 154,647 rows × 12 columns
   ✅ Missing values: 0

🔍 KIỂM TRA DUPLICATES:
   ✅ 📋 Customer (Cleaned): No duplicates
   ✅ 📢 Campaign (Cleaned): No duplicates
   ✅ 📱 Device (Cleaned): No duplicates
   ✅ 🎫 Status (Cleaned): No duplicates
   ✅ 🎬 Ticket History (Cleaned): No duplicates

🔗 KIỂM TRA FOREIGN KEYS:
   ✅ Customer orphan: 0
   ✅ Campaign orphan: 0
   ✅ Device orphan: 0
   ✅ Status orphan: 0

🎉 DỮ LIỆU ĐÃ SẠCH VÀ SẴN SÀNG CHO PHÂN TÍCH!


## 6️⃣ Export Clean Datasets

Lưu các datasets đã làm sạch để sử dụng cho phân tích tiếp theo


In [28]:
# Export Clean Datasets
print("💾 EXPORT CLEAN DATASETS")
print("="*60)

import os

# Tạo thư mục cleaned data nếu chưa có
cleaned_data_dir = '../data/cleaned'
if not os.path.exists(cleaned_data_dir):
    os.makedirs(cleaned_data_dir)
    print(f"📁 Đã tạo thư mục: {cleaned_data_dir}")

try:
    # Export các datasets đã làm sạch
    df_customer.to_csv(f'{cleaned_data_dir}/customer_cleaned.csv', index=False)
    print(f"✅ Đã export: customer_cleaned.csv ({len(df_customer):,} rows)")
    
    df_campaign.to_csv(f'{cleaned_data_dir}/campaign_cleaned.csv', index=False)
    print(f"✅ Đã export: campaign_cleaned.csv ({len(df_campaign):,} rows)")
    
    df_device.to_csv(f'{cleaned_data_dir}/device_detail_cleaned.csv', index=False)
    print(f"✅ Đã export: device_detail_cleaned.csv ({len(df_device):,} rows)")
    
    df_status.to_csv(f'{cleaned_data_dir}/status_detail_cleaned.csv', index=False)
    print(f"✅ Đã export: status_detail_cleaned.csv ({len(df_status):,} rows)")
    
    df_ticket.to_csv(f'{cleaned_data_dir}/ticket_history_cleaned.csv', index=False)
    print(f"✅ Đã export: ticket_history_cleaned.csv ({len(df_ticket):,} rows)")
    
    print(f"\n🎉 HOÀN THÀNH EXPORT CLEAN DATASETS!")
    print(f"📂 Tất cả files đã được lưu trong: {cleaned_data_dir}/")
    
except Exception as e:
    print(f"❌ Lỗi khi export: {e}")
    raise


💾 EXPORT CLEAN DATASETS
📁 Đã tạo thư mục: ../data/cleaned
✅ Đã export: customer_cleaned.csv (131,400 rows)
✅ Đã export: campaign_cleaned.csv (217 rows)
✅ Đã export: device_detail_cleaned.csv (139,901 rows)
✅ Đã export: status_detail_cleaned.csv (8 rows)
✅ Đã export: ticket_history_cleaned.csv (154,647 rows)

🎉 HOÀN THÀNH EXPORT CLEAN DATASETS!
📂 Tất cả files đã được lưu trong: ../data/cleaned/


## 📊 TỔNG KẾT QUÁ TRÌNH LÀM SẠCH DỮ LIỆU

### ✅ **Đã hoàn thành:**

#### **Phase 1 - Data Discovery & Assessment:**
- ✅ Phân tích cấu trúc dữ liệu (426,353 records)
- ✅ Kiểm tra missing values (3 vấn đề)
- ✅ Kiểm tra duplicates (102 duplicate tickets)
- ✅ Kiểm tra data types

#### **Phase 2 - Data Type Conversion & Business Logic:**
- ✅ Chuyển đổi datetime (dob, time)
- ✅ Kiểm tra business logic giá vé (100% đúng)
- ✅ Kiểm tra foreign key relationships

#### **Phase 3 - Data Cleaning & Final Preparation:**
- ✅ Xử lý 102 duplicate tickets
- ✅ Fill 7,139 missing device.model với "Unknown"
- ✅ Drop 1 record có device_number missing
- ✅ Fill 1 missing status.error_group với "no_error"
- ✅ Tạo "No Campaign" record cho campaign_id = 0
- ✅ Drop orphan device records
- ✅ Export clean datasets

### 📈 **Kết quả cuối cùng:**
- **Clean data**: Sẵn sàng cho phân tích
- **No missing values**: Tất cả đã được xử lý
- **No duplicates**: Đã loại bỏ
- **Consistent relationships**: Foreign keys đã được chuẩn hóa
- **Ready for analysis**: Có thể bắt đầu EDA và dashboard

---
